##SETUP

In [21]:
import time
import pandas as pd
# Android ADB
from subprocess import check_output,Popen
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm

import sys
# Start Scrcpy once per restart
if 'started_scrcpy' not in vars():
    started_scrcpy=True
    proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
    time.sleep(6) # <-- sleep for 6''
    proc.terminate() # <-- terminate the process (Scrcpy window can be closed)
# internal
sys.path.append('./src')
import bot_core
import bot_perception

import importlib
[importlib.reload(lib) for lib in [bot_core,bot_perception] ]


# Selected units in deck (order does not matter)
#sel_units= ['crystal.png','chemist.png','monkey.png','dryad.png','hunter.png']
sel_units= ['cauldron.png','bruser.png','harlequin.png','dryad.png','zealot.png']
bot_core.select_units(sel_units,show=False)

bot = bot_core.Bot()

In [7]:
bot_core.select_units(sel_units,show=True)

['alchemist.png', 'archer.png', 'banner.png', 'blade_dancer.png', 'bruser.png', 'catapult.png', 'cauldron.png', 'chemist.png', 'clock.png', 'cold_mage.png', 'corsair.png', 'crystal.png', 'cultist.png', 'cultist_off.png', 'demo.png', 'demon_hunter.png', 'dryad.png', 'empty.png', 'engineer.png', 'executioner.png', 'fire_mage.png', 'flinger.png', 'gargoyle.png', 'harlequin.png', 'hex.png', 'hunter.png', 'inquisitor.png', 'ivy.png', 'knight_statue.png', 'lightning.png', 'meteor.png', 'mime.png', 'monkey.png', 'old', 'old_icon', 'poisoner.png', 'portal_keeper.png', 'portal_mage.png', 'priest.png', 'pyro.png', 'reaper.png', 'rogue.png', 'sentry.png', 'shaman.png', 'sharpshooter.png', 'sharpstone.png', 'summoner.png', 'thunder.png', 'time_keeper.png', 'trapper.png', 'unit_rank', 'vampire.png', 'wind_arch.png', 'witch_doc.png', 'zealot.png']
Chosen:
 ['cauldron.png', 'knight_statue.png', 'harlequin.png', 'dryad.png', 'zealot.png']


## RUN BOT

In [23]:
wait=0
combat = 0
grid_df =None
watch_ad = True
train_ai = False
# Main loop
while(1):
    output = bot.battle_screen(start=False)
    if output[1]=='fighting':
        watch_ad = True 
        wait = 0
        combat+=1
        if combat>50:
            bot.restart_RR()
            combat = 0
            continue
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Upgrade units
                bot.mana_level([5,2,1],hero_power=True)
                # Spawn units
                bot.click(450,1360)
                # Try to merge units
                grid_df,unit_series,merge_series,df_groups,info = bot.try_merge(prev_grid=grid_df)
                clear_output()
                display([f'{combat}, {i+1}/8 {output[1]}, {info}', 'Average age: '+ str(grid_df['Age'].mean().round(2)),grid_df,merge_series,df_groups])
        # Wait until late stage in combat and if consistency is ok, not stagnate save all units for ML model
        if combat==25 and 5<grid_df['Age'].mean()<50 and train_ai:
            bot_perception.add_grid_to_dataset()
    elif output[1]=='home' and watch_ad:
        for i in range(5):
            bot.watch_ads()
        watch_ad = False
    else:
        combat=0
        output = bot.battle_screen(start=True,pve=True,floor=5)
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR(),
            
            wait=0
        print(output[1],'wait count:',wait)

['10, 1/8 fighting, need more units!',
 'Average age: 10.93',
    grid_pos           unit  probability  rank  rank_prob   Age
 0    [0, 0]      empty.png        999.0     0   1.000000   4.0
 1    [0, 1]     bruser.png        587.0     3   1.000000  24.0
 2    [0, 2]     bruser.png        592.0     2   1.000000   7.0
 3    [0, 3]      dryad.png        592.0     1   1.000000   3.0
 4    [0, 4]      empty.png        700.0     1   0.999957   1.0
 5    [1, 0]      dryad.png        589.0     1   0.999998   2.0
 6    [1, 1]     zealot.png        349.0     3   1.000000   4.0
 7    [1, 2]      empty.png        700.0     4   1.000000   0.0
 8    [1, 3]     zealot.png        328.0     1   1.000000   0.0
 9    [1, 4]      dryad.png        218.0     1   1.000000  16.0
 10   [2, 0]  harlequin.png        173.0     3   1.000000  10.0
 11   [2, 1]     bruser.png        621.0     3   1.000000  24.0
 12   [2, 2]     zealot.png        404.0     4   1.000000  40.0
 13   [2, 3]      empty.png        999.0  

In [18]:
# profile bot.try_merge
import cProfile
cProfile.run('bot.try_merge(prev_grid=grid_df)','profile.txt')
# sort by time
import pstats
p = pstats.Stats('profile.txt')
p.sort_stats('time').print_stats(5)


Sat Jul 16 14:07:22 2022    profile.txt

         47663 function calls (47194 primitive calls) in 0.133 seconds

   Ordered by: internal time
   List reduced from 939 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      110    0.051    0.000    0.051    0.000 {method 'detectAndCompute' of 'cv2.Feature2D' objects}
       85    0.033    0.000    0.033    0.000 {imread}
       15    0.003    0.000    0.003    0.000 {Canny}
       50    0.003    0.000    0.003    0.000 {method 'match' of 'cv2.DescriptorMatcher' objects}
       15    0.003    0.000    0.003    0.000 {built-in method _pickle.load}




## View perception

In [46]:
for i in range(200):
    names=bot.scan_grid(new=True)
    time.sleep(2)
    grid_df=bot_perception.grid_status(names,prev_grid=grid_df)
    clear_output()
    display([f'{i} Average age: + {str(grid_df["Age"].mean().round(2))}',grid_df])


['9 Average age: + 3.0',
     grid_id           unit  probability  rank  rank_prob position  Age
 0    icon_0      empty.png        700.0     0   0.610355   [0, 0]  9.0
 1    icon_1      empty.png        700.0     3   0.550593   [0, 1]  1.0
 2    icon_2      empty.png        700.0     3   0.318747   [0, 2]  1.0
 3    icon_3      empty.png        700.0     0   0.547471   [0, 3]  1.0
 4    icon_4      empty.png        700.0     0   0.475457   [0, 4]  9.0
 5    icon_5  harlequin.png        699.0     1   0.236596   [1, 0]  1.0
 6    icon_6      empty.png        999.0     4   0.193083   [1, 1]  1.0
 7    icon_7      empty.png        999.0     0   0.496129   [1, 2]  5.0
 8    icon_8      empty.png        700.0     0   0.970204   [1, 3]  2.0
 9    icon_9      empty.png        700.0     5   0.466748   [1, 4]  1.0
 10  icon_10      empty.png        700.0     4   0.388355   [2, 0]  1.0
 11  icon_11      empty.png        999.0     0   0.235360   [2, 1]  9.0
 12  icon_12      empty.png        999.

KeyboardInterrupt: 

## Quick restart

In [3]:
# Drop co-op game
bot.restart_RR(quick_disconnect=True)

In [2]:
# Restart game
bot.restart_RR()